<a href="https://colab.research.google.com/github/newRevelation/DL/blob/master/copy_of_proj3.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import tensorflow as tf
import numpy as np

import distutils
if distutils.version.LooseVersion(tf.__version__) < '1.14':
    raise Exception('This notebook is compatible with TensorFlow 1.14 or higher, for TensorFlow 1.13 or lower please use the previous version at https://github.com/tensorflow/tpu/blob/r1.13/tools/colab/fashion_mnist.ipynb')


In [0]:
(x_train,y_train),(x_test,y_test) = tf.keras.datasets.cifar10.load_data()

170500096/170498071 [==============================] - 2s 0us/step


In [0]:
'''
x_train = np.dot(x_train[...,:3],[0.299,0.587,0.114])
x_test = np.dot(x_test[...,:3],[0.299,0.587,0.114])
'''

'\nx_train = np.dot(x_train[...,:3],[0.299,0.587,0.114])\nx_test = np.dot(x_test[...,:3],[0.299,0.587,0.114])\n'

In [0]:
'''
# Normalize pixel values to be between -0.5 and 0.5
x_train = (x_train-128.0) / 256.0
x_test = (x_test-128.0) / 256.0
'''
# dfdf

'\n# Normalize pixel values to be between -0.5 and 0.5\nx_train = (x_train-128.0) / 256.0\nx_test = (x_test-128.0) / 256.0\n'

In [0]:
# add empty color dimension
x_train = np.expand_dims(x_train, -1)
x_test = np.expand_dims(x_test, -1)

In [0]:
print(x_train.shape)
print(x_test.shape)

(50000, 32, 32, 3, 1)
(10000, 32, 32, 3, 1)


In [0]:
print(x_test.dtype)

uint8


In [0]:
x_train.shape[1:]

(32, 32, 3, 1)

In [0]:
from tensorflow.keras import models,layers
def create_model():
  model = models.Sequential()
  from keras.layers import LeakyReLU

  model.add(layers.BatchNormalization(input_shape=x_train.shape[1:]))
  model.add(layers.Conv2D(64, (5, 5),strides = 1, padding = 'valid'))
  model.add(layers.LeakyReLU(alpha=0.1))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Conv2D(128, (3, 3),strides = 1, padding = 'valid'))
  model.add(layers.LeakyReLU(alpha=0.1))
  model.add(layers.MaxPooling2D((2, 2)))

  model.add(layers.Conv2D(256, (3, 3), strides = 1, padding = 'valid'))
  model.add(layers.LeakyReLU(alpha=0.1))
  model.add(layers.MaxPooling2D((2, 2)))


  model.add(layers.Flatten())
  model.add(layers.Dense(512))
  model.add(layers.LeakyReLU(alpha=0.1))
  model.add(layers.Dropout(0.3))
  model.add(layers.Dense(10, activation='softmax'))
  model.summary()
  return model

In [0]:
import os

resolver = tf.contrib.cluster_resolver.TPUClusterResolver('grpc://' + os.environ['COLAB_TPU_ADDR'])
tf.contrib.distribute.initialize_tpu_system(resolver)
strategy = tf.contrib.distribute.TPUStrategy(resolver)

The TensorFlow contrib module will not be included in TensorFlow 2.0.
For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
  * https://github.com/tensorflow/io (for I/O related ops)
If you depend on functionality not listed there, please file an issue.

INFO:tensorflow:Initializing the TPU system: 10.84.47.210:8470
INFO:tensorflow:Finished initializing TPU system.
INFO:tensorflow:Querying Tensorflow master (grpc://10.84.47.210:8470) for TPU system metadata.
INFO:tensorflow:Found TPU system:
INFO:tensorflow:*** Num TPU Cores: 8
INFO:tensorflow:*** Num TPU Workers: 1
INFO:tensorflow:*** Num TPU Cores Per Worker: 8
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:CPU:0, CPU, -1, 6828703360382255957)
INFO:tensorflow:*** Available Device: _DeviceAttributes(/job:worker/replica:0/task:0/device:TPU:0, TPU, 17179869184, 14097700158995107303)
IN

In [0]:
with strategy.scope():
  model = create_model()
  model.compile(
      optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3, ),
      loss='sparse_categorical_crossentropy',
      metrics=['acc'])

Instructions for updating:
If using Keras pass *_constraint arguments to layers.


Using TensorFlow backend.


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where


ValueError: ignored

In [0]:
history = model.fit(
    x_train.astype(np.float32),y_train.astype(np.float32),
    epochs=17,steps_per_epoch=100,
    validation_data = (x_test.astype(np.float32),y_test.astype(np.float32)),
    validation_freq=17
    )
#steps_per_epoch=int(np.ceil(x_train.shape[0]/64)),
#validation_data=(x_test,y_test)

model.save_weights('./tpu_model.h5',overwrite=True)
'''
tpu_model.evaluate(x_test,y_test,batch_size=64*8)
'''

Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Epoch 1/17


InvalidArgumentError: ignored